# Laboratorio 7

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import altair as alt

from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score

alt.themes.enable('opaque')
%matplotlib inline

En este laboratorio utilizaremos los mismos datos de diabetes vistos en la clase

In [ ]:
diabetes_X, diabetes_y = datasets.load_diabetes(return_X_y=True, as_frame=True)
diabetes = pd.concat([diabetes_X, diabetes_y], axis=1)
diabetes.head()

## Pregunta 1

(1 pto)

* ¿Por qué la columna de sexo tiene esos valores?
* ¿Cuál es la columna a predecir?
* ¿Crees que es necesario escalar o transformar los datos antes de comenzar el modelamiento?

__Respuesta:__

* Es debido a que se representó el sexo con números, y luego, se centró (teniendo en cuenta el promedio) y se escaló por su desviación estándar multiplicada por la cantidad de muestras.
* target
* No, ya que han sido previamente preprocesados.

## Pregunta 2

(1 pto)

Realiza dos regresiones lineales con todas las _features_, el primer caso incluyendo intercepto y el segundo sin intercepto. Luego obtén la predicción para así calcular el error cuadrático medio y coeficiente de determinación de cada uno de ellos.

In [ ]:
regr_with_incerpet =linear_model.LinearRegression(fit_intercept=True)
regr_with_incerpet.fit(diabetes_X, diabetes_y)

In [ ]:
diabetes_y_pred_with_intercept = regr_with_incerpet.predict(diabetes_X)

In [ ]:
# Coeficientes
print(f"Coefficients: \n{regr_with_incerpet.coef_}\n")
# Intercepto
print(f"Intercept: \n{regr_with_incerpet.intercept_}\n")
# Error cuadrático medio
print(f"Mean squared error: {mean_squared_error(diabetes_y_pred_with_intercept,diabetes_y):.2f}\n")
# Coeficiente de determinación
print(f"Coefficient of determination: {r2_score(diabetes_y_pred_with_intercept,diabetes_y):.2f}")

In [ ]:
regr_without_incerpet =linear_model.LinearRegression(fit_intercept=False)
regr_without_incerpet.fit(diabetes_X, diabetes_y)

In [ ]:
diabetes_y_pred_without_intercept = regr_without_incerpet.predict(diabetes_X)

In [ ]:
# Coeficientes
print(f"Coefficients: \n{regr_without_incerpet.coef_}\n")
# Error cuadrático medio
print(f"Mean squared error: {mean_squared_error(diabetes_y_pred_without_intercept,diabetes_y):.2f}\n")
# Coeficiente de determinación
print(f"Coefficient of determination: {r2_score(diabetes_y_pred_without_intercept,diabetes_y):.2f}")

**Pregunta: ¿Qué tan bueno fue el ajuste del modelo?**

__Respuesta:__ 
* Malo-Regular par el primer caso, teniendo que el valor del coeficiente de determinación está cerca del 0 (según la documentación, por ejemplo, eso sería lo obtenido si se eligiría un modelo constante con la esperanza de y), una comparación entre el Mean squared error y las magnitudes involucradas. 
* Malo , para el segundo,  teniendo en cuenta que es peor que el anterior (mayor Mean squared error, y coeficiente de determinación negativo), es peor que ocupar la media de los y como modelo.
 

## Pregunta 3

(1 pto)

Realizar multiples regresiones lineales utilizando una sola _feature_ a la vez. 

En cada iteración:

- Crea un arreglo `X`con solo una feature filtrando `X`.
- Crea un modelo de regresión lineal con intercepto.
- Ajusta el modelo anterior.
- Genera una predicción con el modelo.
- Calcula e imprime las métricas de la pregunta anterior.

In [ ]:
for col in diabetes_X.columns:
    X_i = diabetes_X[col].values.reshape(-1, 1)
    regr_i = linear_model.LinearRegression(fit_intercept=True)
    regr_i.fit(X_i,diabetes_y)
    diabetes_y_pred_i = regr_i.predict(X_i)
    print(f"Feature: {col}")
    print(f"\tCoefficients: {regr_i.coef_}")
    print(f"\tIntercept: {regr_i.intercept_}")
    print(f"\tMean squared error: {mean_squared_error(diabetes_y_pred_i,diabetes_y):.2f}")
    print(f"\tCoefficient of determination: {r2_score(diabetes_y_pred_i,diabetes_y):.2f}\n")

**Pregunta: Si tuvieras que escoger una sola _feauture_, ¿Cuál sería? ¿Por qué?**

**Respuesta:** Se eligiría bmi, ya que es la que tiene el menor error cuadrado, así como también el mayor coeficiente de determinación.

## Ejercicio 4

(1 pto)

Con la feature escogida en el ejercicio 3 realiza el siguiente gráfico:

- Scatter Plot
- Eje X: Valores de la feature escogida.
- Eje Y: Valores de la columna a predecir (target).
- En color rojo dibuja la recta correspondiente a la regresión lineal (utilizando `intercept_`y `coefs_`).
- Coloca un título adecuado, nombre de los ejes, etc.

Puedes utilizar `matplotlib` o `altair`, el que prefieras.

In [ ]:
regr = linear_model.LinearRegression().fit(diabetes_X["bmi"].values.reshape(-1, 1),diabetes_y)

In [ ]:
prediccion=pd.DataFrame(data=regr.predict(diabetes_X["bmi"].values.reshape(-1, 1))
        .reshape(-1,1),
        columns=["predicción"]
                        )                                         

In [ ]:
source=pd.concat([diabetes_X.loc[:,"bmi"], diabetes_y,prediccion], axis=1)



recta = alt.Chart(source).mark_line(color="red").encode(
    x="bmi:Q",
    y="predicción:Q"
)

scatter= alt.Chart(source,title="target vs. bmi con regresión lineal superpuesta").mark_circle().encode(
    x=alt.X("bmi", type='quantitative'),
    y=alt.Y("target:Q",title="target")
)

scatter+recta